In [ ]:
import tensorflow as tf
import os
import zipfile

# Transfer Learning

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 4.4MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=ea64715e62c76a6620f67eb51ffa08500d7ee6dd706bf205bb47580a3e3c5788
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
  Found existing installation: kaggle 1.5.6
    Uninstalling kaggle-1.5.6:
      Successfully uninstalled kaggle-1.5.6


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

### Importing dataset from Kaggle cat vs dog

In [ ]:
!kaggle competitions download -c dogs-vs-cats -p "/tmp"

 98% 793M/812M [00:13<00:00, 45.1MB/s]
100% 812M/812M [00:13<00:00, 61.1MB/s]


In [ ]:
!kaggle datasets download -d kushleshkumar/cats-and-dogs -p '/tmp'

 93% 185M/199M [00:03<00:00, 56.8MB/s]
100% 199M/199M [00:03<00:00, 62.8MB/s]


In [ ]:
local_zip='/tmp/dogs-vs-cats.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
local_zip='/tmp/train.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
local_zip='/tmp/cats-and-dogs.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
!mkdir "/tmp/train/cats"
!mkdir "/tmp/train/dogs"

In [ ]:
!mv "/tmp/train/"dog* "/tmp/train/dogs"
!mv "/tmp/train/"cat* "/tmp/train/cats"

mv: cannot move '/tmp/train/dogs' to a subdirectory of itself, '/tmp/train/dogs/dogs'
mv: cannot move '/tmp/train/cats' to a subdirectory of itself, '/tmp/train/cats/cats'


In [ ]:
!ls -al "/tmp/train"

total 1496
drwxr-xr-x 4 root root 765952 Jul 13 14:33 .
drwxrwxrwt 1 root root   4096 Jul 13 14:33 ..
drwxr-xr-x 2 root root 389120 Jul 13 14:33 cats
drwxr-xr-x 2 root root 372736 Jul 13 14:33 dogs


## Making Train Set and Validation Set

In [ ]:
train_dir= "/tmp/train"
Val_dir="/tmp/KK cats_and_dogs_filtered/train"


# Directory with our training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat/dog pictures
validation_cats_dir = os.path.join(Val_dir, 'cats')
validation_dogs_dir = os.path.join(Val_dir, 'dogs')

## **Starting Transfer Learning**

In [ ]:
#Base Model is VGG19
base_model=tf.keras.applications.VGG19(
    include_top=False, # Load weights pre-trained on ImageNet.
    weights="imagenet",
    input_shape=(150, 150, 3)
)
#Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = tf.keras.Input(shape=(150, 150, 3))

x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x) #Pooling Layer
x = tf.keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
vgg19 (Model)                (None, 4, 4, 512)         20024384  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 20,024,897
Trainable params: 513
Non-trainable params: 20,024,384
_________________________________________________________________


In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

##***Using Image Data Generator to train, Validation directory***

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(Val_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))


Found 25000 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


##**Train top layer**

In [ ]:
trainer=model.fit(train_generator,
                  steps_per_epoch=125,
                  validation_data=validation_generator,
                  validation_steps=25,
                  epochs=15,
                  verbose=1)

Epoch 1/15
125/125 [==============================] - 23s 187ms/step - loss: 0.5486 - accuracy: 0.8060 - val_loss: 0.4465 - val_accuracy: 0.8340
Epoch 2/15
125/125 [==============================] - 24s 191ms/step - loss: 0.6243 - accuracy: 0.8128 - val_loss: 0.4429 - val_accuracy: 0.8360
Epoch 3/15
125/125 [==============================] - 23s 187ms/step - loss: 0.5360 - accuracy: 0.8112 - val_loss: 0.4522 - val_accuracy: 0.8440
Epoch 4/15
125/125 [==============================] - 24s 189ms/step - loss: 0.5062 - accuracy: 0.8276 - val_loss: 0.5498 - val_accuracy: 0.8260
Epoch 5/15
125/125 [==============================] - 23s 187ms/step - loss: 0.4950 - accuracy: 0.8292 - val_loss: 0.4406 - val_accuracy: 0.8660
Epoch 6/15
125/125 [==============================] - 24s 194ms/step - loss: 0.4359 - accuracy: 0.8320 - val_loss: 0.5317 - val_accuracy: 0.8380
Epoch 7/15
125/125 [==============================] - 23s 186ms/step - loss: 0.4607 - accuracy: 0.8320 - val_loss: 0.4362 - val_ac

#*Predicting uploaded image*

In [ ]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path='/content/' + fn
  img=image.load_img(path, target_size=(150, 150))
  
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])
  
  if classes[0]>0:
    print(fn + " is a dog")
    
  else:
    print(fn + " is a cat")
 

Saving images (1).jfif to images (1).jfif
[-54.25614]
images (1).jfif is a cat
